코드

In [1]:
import warnings
warnings.filterwarnings('ignore')

from glob import glob
import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2

import os
import timm
import random

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torchvision.transforms as transforms
from sklearn.metrics import f1_score, accuracy_score
import time

from sklearn.model_selection import StratifiedKFold
device = torch.device('cuda')

In [2]:
train_png = sorted(glob('./open/train/*.png'))
test_png = sorted(glob('./open/test/*.png'))

In [3]:
len(train_png), len(test_png)

(4277, 2154)

In [3]:
train_y = pd.read_csv("./open/train_df.csv")

train_labels = train_y["label"]

label_unique = sorted(np.unique(train_labels))
label_unique = {key:value for key,value in zip(label_unique, range(len(label_unique)))}

train_labels = [label_unique[k] for k in train_labels]

In [4]:
def img_load(path):
    img = cv2.imread(path)[:,:,::-1]
    img = cv2.resize(img, (384, 384),interpolation = cv2.INTER_AREA)
    return img

In [5]:
train_imgs = [img_load(m) for m in tqdm(train_png)]
test_imgs = [img_load(n) for n in tqdm(test_png)]

  4%|▍         | 163/4277 [00:03<01:33, 43.92it/s]

KeyboardInterrupt: 

In [9]:
np.save('./save/train_imgs_384', np.array(train_imgs))
np.save('./save/test_imgs_384', np.array(test_imgs))

In [6]:
train_imgs = np.load('./save/train_imgs_384.npy')
test_imgs = np.load('./save/test_imgs_384.npy')

In [7]:
meanRGB = [np.mean(x, axis=(0,1)) for x in train_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in train_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("train 평균",meanR, meanG, meanB)
print("train 표준편차",stdR, stdG, stdB)

train 평균 0.4330380901867049 0.4034575319032911 0.39415050509784405
train 표준편차 0.1815717110252788 0.17403455556798705 0.16323395055036488


In [8]:
meanRGB = [np.mean(x, axis=(0,1)) for x in test_imgs]
stdRGB = [np.std(x, axis=(0,1)) for x in test_imgs]

meanR = np.mean([m[0] for m in meanRGB])/255
meanG = np.mean([m[1] for m in meanRGB])/255
meanB = np.mean([m[2] for m in meanRGB])/255

stdR = np.mean([s[0] for s in stdRGB])/255
stdG = np.mean([s[1] for s in stdRGB])/255
stdB = np.mean([s[2] for s in stdRGB])/255

print("test 평균",meanR, meanG, meanB)
print("test 표준편차",stdR, stdG, stdB)

test 평균 0.41825619520929724 0.3931011906330291 0.386631764639131
test 표준편차 0.19505524270747931 0.19005280951759498 0.18053225852732663


In [7]:
class Custom_dataset(Dataset):
    def __init__(self, img_paths, labels, mode='train'):
        self.img_paths = img_paths
        self.labels = labels
        self.mode=mode
    def __len__(self):
        return len(self.img_paths)
    def __getitem__(self, idx):
        img = self.img_paths[idx]
        if self.mode == 'train':
          train_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.433038, 0.403458, 0.394151],
                                     std = [0.181572, 0.174035, 0.163234]),
                transforms.RandomAffine((-45, 45)),
                
            ])
          img = train_transform(img)
        if self.mode == 'test':
          test_transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = [0.418256, 0.393101, 0.386632],
                                     std = [0.195055, 0.190053, 0.185323])
            ])
          img = test_transform(img)

        
        label = self.labels[idx]
        return img, label
    
class Network(nn.Module):
    def __init__(self,mode = 'train'):
        super(Network, self).__init__()
        self.mode = mode
        if self.mode == 'train':
          self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88, drop_path_rate = 0.2)
        if self.mode == 'test':
          self.model = timm.create_model('efficientnet_b3', pretrained=True, num_classes=88, drop_path_rate = 0)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [8]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

In [9]:
def main(seed = 2022):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True
    
main(2022)

In [10]:
import gc

cv = StratifiedKFold(n_splits = 5, random_state = 2022,shuffle=True)
batch_size = 8
epochs = 70
pred_ensemble = []


for idx, (train_idx, val_idx) in enumerate(cv.split(train_imgs, np.array(train_labels))):
    print("----------fold_{} start!----------".format(idx))
    t_imgs, val_imgs = train_imgs[train_idx],  train_imgs[val_idx]
    t_labels, val_labels = np.array(train_labels)[train_idx], np.array(train_labels)[val_idx]

    # Train
    train_dataset = Custom_dataset(np.array(t_imgs), np.array(t_labels), mode='train')
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

    # Val
    val_dataset = Custom_dataset(np.array(val_imgs), np.array(val_labels), mode='test')
    val_loader = DataLoader(val_dataset, shuffle=True, batch_size=batch_size)

    gc.collect()
    torch.cuda.empty_cache()
    best=0

    model = Network().to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4, weight_decay = 2e-2)
    criterion = nn.CrossEntropyLoss()
    scaler = torch.cuda.amp.GradScaler()  

    best_f1 = 0
    early_stopping = 0
    for epoch in range(epochs):
        start=time.time()
        train_loss = 0
        train_pred=[]
        train_y=[]
        model.train()
        for batch in (train_loader):
            optimizer.zero_grad()
            x = torch.tensor(batch[0], dtype=torch.float32, device=device)
            y = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred = model(x)
            loss = criterion(pred, y)


            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            
            train_loss += loss.item()/len(train_loader)
            train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
            train_y += y.detach().cpu().numpy().tolist()
        train_f1 = score_function(train_y, train_pred)
        state_dict= model.state_dict()
        model.eval()
        with torch.no_grad():
            val_loss = 0 
            val_pred = []
            val_y = []
        

        for batch in (val_loader):
            x_val = torch.tensor(batch[0], dtype = torch.float32, device = device)
            y_val = torch.tensor(batch[1], dtype=torch.long, device=device)
            with torch.cuda.amp.autocast():
                pred_val = model(x_val)
            loss_val = criterion(pred_val, y_val)

            val_loss += loss_val.item()/len(val_loader)
            val_pred += pred_val.argmax(1).detach().cpu().numpy().tolist()
            val_y += y_val.detach().cpu().numpy().tolist()
        val_f1 = score_function(val_y, val_pred)

        if val_f1 > best_f1:
            best_epoch = epoch
            best_loss = val_loss
            best_f1 = val_f1
            early_stopping = 0

            torch.save({'epoch':epoch,
                        'state_dict':state_dict,
                        'optimizer': optimizer.state_dict(),
                        'scaler': scaler.state_dict(),
                }, './save/best_model_{}.pth'.format(idx))
            print('-----------------SAVE:{} epoch----------------'.format(best_epoch+1))
        else:
            early_stopping += 1

                # Early Stopping
        if early_stopping == 20:
            TIME = time.time() - start
            print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
            print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
            print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')
            break

        TIME = time.time() - start
        print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
        print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')
        print(f'Val    loss : {val_loss:.5f}    f1 : {val_f1:.5f}')

----------fold_0 start!----------


  4%|▍         | 163/4277 [00:20<01:33, 43.92it/s]

-----------------SAVE:1 epoch----------------
epoch : 1/70    time : 114s/7898s
TRAIN    loss : 1.07745    f1 : 0.19511
Val    loss : 0.58840    f1 : 0.31208
-----------------SAVE:2 epoch----------------
epoch : 2/70    time : 102s/6932s
TRAIN    loss : 0.54709    f1 : 0.38922
Val    loss : 0.40714    f1 : 0.47081
-----------------SAVE:3 epoch----------------
epoch : 3/70    time : 103s/6920s
TRAIN    loss : 0.36941    f1 : 0.52961
Val    loss : 0.32156    f1 : 0.57020
-----------------SAVE:4 epoch----------------
epoch : 4/70    time : 105s/6935s
TRAIN    loss : 0.28453    f1 : 0.63502
Val    loss : 0.29064    f1 : 0.63231
-----------------SAVE:5 epoch----------------
epoch : 5/70    time : 106s/6867s
TRAIN    loss : 0.21027    f1 : 0.72595
Val    loss : 0.40196    f1 : 0.63297
-----------------SAVE:6 epoch----------------
epoch : 6/70    time : 106s/6770s
TRAIN    loss : 0.18390    f1 : 0.76712
Val    loss : 0.24777    f1 : 0.74168
-----------------SAVE:7 epoch----------------
epoch 

In [11]:
pred_ensemble = []
batch_size = 32
# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

for i in range(5):
  model_test = Network(mode = 'test').to(device)
  model_test.load_state_dict(torch.load(('./save/best_model_{}.pth'.format(i)))['state_dict'])
  model_test.eval()
  pred_prob = []
  with torch.no_grad():
      for batch in (test_loader):
          x = torch.tensor(batch[0], dtype = torch.float32, device = device)
          with torch.cuda.amp.autocast():
              pred = model_test(x)
              pred_prob.extend(pred.detach().cpu().numpy())
      pred_ensemble.append(pred_prob)

In [12]:
len(pred_ensemble)

5

In [13]:
pred = (np.array(pred_ensemble[0])+ np.array(pred_ensemble[1])+ np.array(pred_ensemble[3]) + np.array(pred_ensemble[4]) )/4
f_pred = np.array(pred).argmax(1).tolist()

In [14]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

In [15]:
submission = pd.read_csv("./open/sample_submission.csv")

submission["label"] = f_result

submission

,index,label
0,0,tile-glue_strip
1,1,grid-good
2,2,transistor-good
3,3,tile-gray_stroke
4,4,tile-good
...,...,...
2149,2149,tile-gray_stroke
2150,2150,screw-good
2151,2151,grid-good
2152,2152,cable-good


In [ ]:
submission.to_csv("./open/b3_norm_epoch70.csv", index = False)

모델 학습

In [ ]:
batch_size = 32
epochs = 30

# Train
train_dataset = Custom_dataset(np.array(train_imgs), np.array(train_labels), mode='train')
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

# Test
test_dataset = Custom_dataset(np.array(test_imgs), np.array(["tmp"]*len(test_imgs)), mode='test')
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
def score_function(real, pred):
    score = f1_score(real, pred, average="macro")
    return score

model = Network().to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5, weight_decay = 1e-3)
criterion = nn.CrossEntropyLoss()
scaler = torch.cuda.amp.GradScaler() 

batch_size = 32
epochs = 30

best=0
for epoch in range(epochs):
    start=time.time()
    train_loss = 0
    train_pred=[]
    train_y=[]
    model.train()
    for batch in (train_loader):
        optimizer.zero_grad()
        x = torch.tensor(batch[0], dtype=torch.float32, device=device)
        y = torch.tensor(batch[1], dtype=torch.long, device=device)
        with torch.cuda.amp.autocast():
            pred = model(x)
        loss = criterion(pred, y)


        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        train_loss += loss.item()/len(train_loader)
        train_pred += pred.argmax(1).detach().cpu().numpy().tolist()
        train_y += y.detach().cpu().numpy().tolist()
        
    
    train_f1 = score_function(train_y, train_pred)

    TIME = time.time() - start
    print(f'epoch : {epoch+1}/{epochs}    time : {TIME:.0f}s/{TIME*(epochs-epoch-1):.0f}s')
    print(f'TRAIN    loss : {train_loss:.5f}    f1 : {train_f1:.5f}')

추론

In [ ]:
model.eval()
f_pred = []
pred_prob = []

with torch.no_grad():
    for batch in (test_loader):
        x = torch.tensor(batch[0], dtype = torch.float32, device = device)
        with torch.cuda.amp.autocast():
            pred = model(x)
            pred_prob.extend(pred.detach().cpu().numpy())
        f_pred.extend(pred.argmax(1).detach().cpu().numpy().tolist())

In [ ]:
label_decoder = {val:key for key, val in label_unique.items()}

f_result = [label_decoder[result] for result in f_pred]

제출물 생성

In [ ]:
submission = pd.read_csv("./open/sample_submission.csv")

submission["label"] = f_result

submission

In [ ]:
submission.to_csv("./open/b3_norm.csv", index = False)